In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime, timedelta

In [2]:
job_posting = pd.read_csv('Job Posting.csv')
job_posting.head()

,Website Domain,Ticker,Job Opening Title,Job Opening URL,First Seen At,Last Seen At,Location,Location Data,Category,Seniority,...,Description,Salary,Salary Data,Contract Types,Job Status,Job Language,Job Last Processed At,O*NET Code,O*NET Family,O*NET Occupation Name
0,bosch.com,NaN,IN_RBAI_Assistant Manager_Dispensing Process E...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-05-29T19:59:45Z,2024-07-31T14:35:44Z,"Indiana, United States","[{""city"":null,""state"":""Indiana"",""zip_code"":nul...","engineering, management, support",manager,...,**IN\_RBAI\_Assistant Manager\_Dispensing Proc...,NaN,"{""salary_low"":null,""salary_high"":null,""salary_...",full time,closed,en,2024-08-02T14:47:55Z,43-1011.00,Office and Administrative Support,First-Line Supervisors of Office and Administr...
1,bosch.com,NaN,Professional Internship: Hardware Development ...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-05-04T01:00:12Z,2024-07-29T17:46:16Z,"Delaware, United States","[{""city"":null,""state"":""Delaware"",""zip_code"":nu...",internship,non_manager,...,**Professional Internship: Hardware Developmen...,NaN,"{""salary_low"":null,""salary_high"":null,""salary_...","full time, internship, m/f",closed,en,2024-07-31T17:50:07Z,17-2061.00,Architecture and Engineering,Computer Hardware Engineers
2,zf.com,NaN,Process Expert BMS Production,https://jobs.zf.com/job/Shenyang-Process-Exper...,2024-04-19T06:47:24Z,2024-05-16T02:25:08Z,China,"[{""city"":null,""state"":null,""zip_code"":null,""co...",engineering,non_manager,...,ZF is a global technology company supplying sy...,NaN,"{""salary_low"":null,""salary_high"":null,""salary_...",NaN,closed,en,2024-05-18T02:32:04Z,51-9141.00,Production,Semiconductor Processing Technicians
3,bosch.com,NaN,DevOps Developer with Python for ADAS Computin...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-08-16T10:20:37Z,2024-08-22T11:14:49Z,Romania,"[{""city"":null,""state"":null,""zip_code"":null,""co...","information_technology, software_development",non_manager,...,**DevOps Developer with Python for ADAS Comput...,NaN,"{""salary_low"":null,""salary_high"":null,""salary_...",full time,closed,en,2024-08-23T00:33:30Z,15-1252.00,Computer and Mathematical,Software Developers
4,bosch.com,NaN,Senior Engineer Sales - Video Systems and Solu...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-07-01T17:31:20Z,2024-08-01T05:11:33Z,India,"[{""city"":null,""state"":null,""zip_code"":null,""co...","engineering, sales",non_manager,...,**Senior Engineer Sales - Video Systems and So...,NaN,"{""salary_low"":null,""salary_high"":null,""salary_...",full time,closed,en,2024-08-02T19:03:16Z,41-9031.00,Sales and Related,Sales Engineers


In [3]:
job_posting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9919 entries, 0 to 9918
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Website Domain         9919 non-null   object 
 1   Ticker                 0 non-null      float64
 2   Job Opening Title      9919 non-null   object 
 3   Job Opening URL        9919 non-null   object 
 4   First Seen At          9919 non-null   object 
 5   Last Seen At           9919 non-null   object 
 6   Location               9508 non-null   object 
 7   Location Data          9919 non-null   object 
 8   Category               8250 non-null   object 
 9   Seniority              9919 non-null   object 
 10  Keywords               7646 non-null   object 
 11  Description            9807 non-null   object 
 12  Salary                 576 non-null    object 
 13  Salary Data            9919 non-null   object 
 14  Contract Types         8004 non-null   object 
 15  Job 

In [4]:
job_posting.describe()

,Ticker
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [5]:
job_posting.Salary.value_counts()

Salary
946,00 EUR                  35
$50                         30
$78 BILLION EUROS A YEAR    18
$19.50                      16
$22.00                      12
                            ..
$160,000 - $174,500          1
$36/hr                       1
$43-68/hourly                1
od 1.800 Û brutto            1
2 191 Û                      1
Name: count, Length: 257, dtype: int64

In [6]:
job_posting.duplicated().sum()

0

In [7]:
job_posting.isnull().sum()

Website Domain              0
Ticker                   9919
Job Opening Title           0
Job Opening URL             0
First Seen At               0
Last Seen At                0
Location                  411
Location Data               0
Category                 1669
Seniority                   0
Keywords                 2273
Description               112
Salary                   9343
Salary Data                 0
Contract Types           1915
Job Status               3147
Job Language                2
Job Last Processed At       0
O*NET Code                  3
O*NET Family                3
O*NET Occupation Name       3
dtype: int64

## EDA `Salary Data`

In [8]:
job_posting['Salary Data']

0       {"salary_low":null,"salary_high":null,"salary_...
1       {"salary_low":null,"salary_high":null,"salary_...
2       {"salary_low":null,"salary_high":null,"salary_...
3       {"salary_low":null,"salary_high":null,"salary_...
4       {"salary_low":null,"salary_high":null,"salary_...
                              ...                        
9914    {"salary_low":null,"salary_high":null,"salary_...
9915    {"salary_low":null,"salary_high":null,"salary_...
9916    {"salary_low":null,"salary_high":null,"salary_...
9917    {"salary_low":null,"salary_high":null,"salary_...
9918    {"salary_low":null,"salary_high":null,"salary_...
Name: Salary Data, Length: 9919, dtype: object

In [9]:
job_posting['Salary Data'] = job_posting['Salary Data'].apply(json.loads)

salary_data = pd.json_normalize(job_posting['Salary Data'])

job_posting = job_posting.drop(columns=['Salary Data']).join(salary_data)

print(job_posting)

     Website Domain  Ticker  \
0         bosch.com     NaN   
1         bosch.com     NaN   
2            zf.com     NaN   
3         bosch.com     NaN   
4         bosch.com     NaN   
...             ...     ...   
9914      bosch.com     NaN   
9915      bosch.com     NaN   
9916      bosch.com     NaN   
9917      bosch.com     NaN   
9918         zf.com     NaN   

                                      Job Opening Title  \
0     IN_RBAI_Assistant Manager_Dispensing Process E...   
1     Professional Internship: Hardware Development ...   
2                         Process Expert BMS Production   
3     DevOps Developer with Python for ADAS Computin...   
4     Senior Engineer Sales - Video Systems and Solu...   
...                                                 ...   
9914                                 Apprentice Program   
9915  Environmental, Health and Safety (EHS) Special...   
9916          Technology & Quality Expert in Purchasing   
9917  Extracurricular Internship: Tes

In [10]:
job_posting.columns

Index(['Website Domain', 'Ticker', 'Job Opening Title', 'Job Opening URL',
       'First Seen At', 'Last Seen At', 'Location', 'Location Data',
       'Category', 'Seniority', 'Keywords', 'Description', 'Salary',
       'Contract Types', 'Job Status', 'Job Language', 'Job Last Processed At',
       'O*NET Code', 'O*NET Family', 'O*NET Occupation Name', 'salary_low',
       'salary_high', 'salary_currency', 'salary_low_usd', 'salary_high_usd',
       'salary_time_unit'],
      dtype='object')

In [11]:
salary_job = job_posting.loc[:, 'salary_low':'salary_time_unit']
print(salary_job.isnull().sum())

salary_low          9485
salary_high         9485
salary_currency     9485
salary_low_usd      9485
salary_high_usd     9485
salary_time_unit    9485
dtype: int64


In [12]:
print(salary_job.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9919 entries, 0 to 9918
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   salary_low        434 non-null    float64
 1   salary_high       434 non-null    float64
 2   salary_currency   434 non-null    object 
 3   salary_low_usd    434 non-null    float64
 4   salary_high_usd   434 non-null    float64
 5   salary_time_unit  434 non-null    object 
dtypes: float64(4), object(2)
memory usage: 465.1+ KB
None


In [13]:
salary_job

,salary_low,salary_high,salary_currency,salary_low_usd,salary_high_usd,salary_time_unit
0,NaN,NaN,None,NaN,NaN,None
1,NaN,NaN,None,NaN,NaN,None
2,NaN,NaN,None,NaN,NaN,None
3,NaN,NaN,None,NaN,NaN,None
4,NaN,NaN,None,NaN,NaN,None
...,...,...,...,...,...,...
9914,NaN,NaN,None,NaN,NaN,None
9915,NaN,NaN,None,NaN,NaN,None
9916,NaN,NaN,None,NaN,NaN,None
9917,NaN,NaN,None,NaN,NaN,None


In [14]:
null_values = salary_job.isnull().sum().sum()
not_null_values = salary_job.notnull().sum().sum()
total_data = salary_job.size
total_missing = salary_job.isnull().sum().sum()

percentage = (total_missing / total_data) * 100

print(f'Null values in data\t: {null_values}')
print(f'Not null values in data\t: {not_null_values}')
print(f'Dimension of Data\t: {total_data}')
print(f'Percentage missing values : {percentage:.2f}%')

Null values in data	: 56910
Not null values in data	: 2604
Dimension of Data	: 59514
Percentage missing values : 95.62%


#### I decide to drop column `salary_low,	salary_high,	salary_currency,	salary_low_usd,	salary_high_usd,	salary_time_unit` cause the ratio missing values is very high

In [15]:
job_posting.drop(columns=['salary_low', 'salary_high', 'salary_currency', 'salary_low_usd', 'salary_high_usd', 'salary_time_unit'], axis=1, inplace=True)

#### and then i drop column Ticker 

In [16]:
job_posting.drop('Ticker', axis=1, inplace=True)

In [17]:
job_posting.head()

,Website Domain,Job Opening Title,Job Opening URL,First Seen At,Last Seen At,Location,Location Data,Category,Seniority,Keywords,Description,Salary,Contract Types,Job Status,Job Language,Job Last Processed At,O*NET Code,O*NET Family,O*NET Occupation Name
0,bosch.com,IN_RBAI_Assistant Manager_Dispensing Process E...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-05-29T19:59:45Z,2024-07-31T14:35:44Z,"Indiana, United States","[{""city"":null,""state"":""Indiana"",""zip_code"":nul...","engineering, management, support",manager,NaN,**IN\_RBAI\_Assistant Manager\_Dispensing Proc...,NaN,full time,closed,en,2024-08-02T14:47:55Z,43-1011.00,Office and Administrative Support,First-Line Supervisors of Office and Administr...
1,bosch.com,Professional Internship: Hardware Development ...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-05-04T01:00:12Z,2024-07-29T17:46:16Z,"Delaware, United States","[{""city"":null,""state"":""Delaware"",""zip_code"":nu...",internship,non_manager,Scrum,**Professional Internship: Hardware Developmen...,NaN,"full time, internship, m/f",closed,en,2024-07-31T17:50:07Z,17-2061.00,Architecture and Engineering,Computer Hardware Engineers
2,zf.com,Process Expert BMS Production,https://jobs.zf.com/job/Shenyang-Process-Exper...,2024-04-19T06:47:24Z,2024-05-16T02:25:08Z,China,"[{""city"":null,""state"":null,""zip_code"":null,""co...",engineering,non_manager,SAP,ZF is a global technology company supplying sy...,NaN,NaN,closed,en,2024-05-18T02:32:04Z,51-9141.00,Production,Semiconductor Processing Technicians
3,bosch.com,DevOps Developer with Python for ADAS Computin...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-08-16T10:20:37Z,2024-08-22T11:14:49Z,Romania,"[{""city"":null,""state"":null,""zip_code"":null,""co...","information_technology, software_development",non_manager,"GitHub, Jenkins, Growth, C++, Linux, Python, M...",**DevOps Developer with Python for ADAS Comput...,NaN,full time,closed,en,2024-08-23T00:33:30Z,15-1252.00,Computer and Mathematical,Software Developers
4,bosch.com,Senior Engineer Sales - Video Systems and Solu...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-07-01T17:31:20Z,2024-08-01T05:11:33Z,India,"[{""city"":null,""state"":null,""zip_code"":null,""co...","engineering, sales",non_manager,Business Development,**Senior Engineer Sales - Video Systems and So...,NaN,full time,closed,en,2024-08-02T19:03:16Z,41-9031.00,Sales and Related,Sales Engineers


#### Split column `Location Data`

In [18]:
def extract_loc_data(location):
    if pd.isna(location):  # Cek jika data NaN
        return {}  # Kembalikan dictionary kosong untuk NaN
    try:
        location = json.loads(location)
        if isinstance(location, list) and len(location) > 0:
            # Kembalikan dictionary pertama jika berbentuk list
            return location[0]
        else:
            return {}  # Kembalikan dictionary kosong jika tidak ada data valid
    except (json.JSONDecodeError, TypeError):
        return {}  # Kembalikan dictionary kosong jika terjadi error


loc_data = job_posting['Location Data'].apply(extract_loc_data)
loc_df = pd.json_normalize(loc_data)

# Gabungkan DataFrame baru dengan job_posting
job_posting = pd.concat([job_posting, loc_df], axis=1)

#### Drop column Loaction data, cause we have column `Location Data`

In [19]:
job_posting.drop('Location Data', axis=1, inplace=True)

In [20]:
job_posting.head()

,Website Domain,Job Opening Title,Job Opening URL,First Seen At,Last Seen At,Location,Category,Seniority,Keywords,Description,...,O*NET Code,O*NET Family,O*NET Occupation Name,city,state,zip_code,country,region,continent,fuzzy_match
0,bosch.com,IN_RBAI_Assistant Manager_Dispensing Process E...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-05-29T19:59:45Z,2024-07-31T14:35:44Z,"Indiana, United States","engineering, management, support",manager,NaN,**IN\_RBAI\_Assistant Manager\_Dispensing Proc...,...,43-1011.00,Office and Administrative Support,First-Line Supervisors of Office and Administr...,None,Indiana,None,United States,None,None,False
1,bosch.com,Professional Internship: Hardware Development ...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-05-04T01:00:12Z,2024-07-29T17:46:16Z,"Delaware, United States",internship,non_manager,Scrum,**Professional Internship: Hardware Developmen...,...,17-2061.00,Architecture and Engineering,Computer Hardware Engineers,None,Delaware,None,United States,None,None,False
2,zf.com,Process Expert BMS Production,https://jobs.zf.com/job/Shenyang-Process-Exper...,2024-04-19T06:47:24Z,2024-05-16T02:25:08Z,China,engineering,non_manager,SAP,ZF is a global technology company supplying sy...,...,51-9141.00,Production,Semiconductor Processing Technicians,None,None,None,China,None,None,False
3,bosch.com,DevOps Developer with Python for ADAS Computin...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-08-16T10:20:37Z,2024-08-22T11:14:49Z,Romania,"information_technology, software_development",non_manager,"GitHub, Jenkins, Growth, C++, Linux, Python, M...",**DevOps Developer with Python for ADAS Comput...,...,15-1252.00,Computer and Mathematical,Software Developers,None,None,None,Romania,None,None,False
4,bosch.com,Senior Engineer Sales - Video Systems and Solu...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-07-01T17:31:20Z,2024-08-01T05:11:33Z,India,"engineering, sales",non_manager,Business Development,**Senior Engineer Sales - Video Systems and So...,...,41-9031.00,Sales and Related,Sales Engineers,None,None,None,India,None,None,False


In [21]:
anjay = job_posting.loc[:, 'city':'fuzzy_match']
anjay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9919 entries, 0 to 9918
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   city         6281 non-null   object
 1   state        2450 non-null   object
 2   zip_code     1045 non-null   object
 3   country      9449 non-null   object
 4   region       21 non-null     object
 5   continent    33 non-null     object
 6   fuzzy_match  9503 non-null   object
dtypes: object(7)
memory usage: 542.6+ KB


#### `City`

In [22]:
null_values = anjay['city'].isnull().sum().sum()
not_null_values = anjay['city'].notnull().sum().sum()
total_data = anjay['city'].size
total_missing = anjay['city'].isnull().sum().sum()

percentage = (total_missing / total_data) * 100

print(f'Null values in data\t: {null_values}')
print(f'Not null values in data\t: {not_null_values}')
print(f'Dimension of Data\t: {total_data}')
print(f'Percentage missing values : {percentage:.2f}%')

Null values in data	: 3638
Not null values in data	: 6281
Dimension of Data	: 9919
Percentage missing values : 36.68%


#### `state`

In [23]:
null_values = anjay['state'].isnull().sum().sum()
not_null_values = anjay['state'].notnull().sum().sum()
total_data = anjay['state'].size
total_missing = anjay['state'].isnull().sum().sum()

percentage = (total_missing / total_data) * 100

print(f'Null values in data\t: {null_values}')
print(f'Not null values in data\t: {not_null_values}')
print(f'Dimension of Data\t: {total_data}')
print(f'Percentage missing values : {percentage:.2f}%')

Null values in data	: 7469
Not null values in data	: 2450
Dimension of Data	: 9919
Percentage missing values : 75.30%


#### `Zip_Code`

In [24]:
null_values = anjay['zip_code'].isnull().sum().sum()
not_null_values = anjay['zip_code'].notnull().sum().sum()
total_data = anjay['zip_code'].size
total_missing = anjay['zip_code'].isnull().sum().sum()

percentage = (total_missing / total_data) * 100

print(f'Null values in data\t: {null_values}')
print(f'Not null values in data\t: {not_null_values}')
print(f'Dimension of Data\t: {total_data}')
print(f'Percentage missing values : {percentage:.2f}%')

Null values in data	: 8874
Not null values in data	: 1045
Dimension of Data	: 9919
Percentage missing values : 89.46%


#### `Country`

In [25]:
null_values = anjay['country'].isnull().sum().sum()
not_null_values = anjay['country'].notnull().sum().sum()
total_data = anjay['country'].size
total_missing = anjay['country'].isnull().sum().sum()

percentage = (total_missing / total_data) * 100

print(f'Null values in data\t: {null_values}')
print(f'Not null values in data\t: {not_null_values}')
print(f'Dimension of Data\t: {total_data}')
print(f'Percentage missing values : {percentage:.2f}%')

Null values in data	: 470
Not null values in data	: 9449
Dimension of Data	: 9919
Percentage missing values : 4.74%


#### `Region`

In [26]:
null_values = anjay['region'].isnull().sum().sum()
not_null_values = anjay['region'].notnull().sum().sum()
total_data = anjay['region'].size
total_missing = anjay['region'].isnull().sum().sum()

percentage = (total_missing / total_data) * 100

print(f'Null values in data\t: {null_values}')
print(f'Not null values in data\t: {not_null_values}')
print(f'Dimension of Data\t: {total_data}')
print(f'Percentage missing values : {percentage:.2f}%')

Null values in data	: 9898
Not null values in data	: 21
Dimension of Data	: 9919
Percentage missing values : 99.79%


#### `Continent`

In [27]:
null_values = anjay['continent'].isnull().sum().sum()
not_null_values = anjay['continent'].notnull().sum().sum()
total_data = anjay['continent'].size
total_missing = anjay['continent'].isnull().sum().sum()

percentage = (total_missing / total_data) * 100

print(f'Null values in data\t: {null_values}')
print(f'Not null values in data\t: { not_null_values}')
print(f'Dimension of Data\t: {total_data}')
print(f'Percentage missing values : {percentage:.2f}%')

Null values in data	: 9886
Not null values in data	: 33
Dimension of Data	: 9919
Percentage missing values : 99.67%


####  `fuzzy_match`

In [28]:
null_values = anjay['fuzzy_match'].isnull().sum().sum()
not_null_values = anjay['fuzzy_match'].notnull().sum().sum()
total_data = anjay['fuzzy_match'].size
total_missing = anjay['fuzzy_match'].isnull().sum().sum()

percentage = (total_missing / total_data) * 100

print(f'Null values in data\t: {null_values}')
print(f'Not null values in data\t: {not_null_values}')
print(f'Dimension of Data\t: {total_data}')
print(f'Percentage missing values : {percentage:.2f}%')

Null values in data	: 416
Not null values in data	: 9503
Dimension of Data	: 9919
Percentage missing values : 4.19%


In [29]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)

In [30]:
anjay['region'].unique()

array([None, nan, 'South America', 'Northern America', 'Southern Africa'],
      dtype=object)

In [31]:
anjay['country'].unique()

array(['United States', 'China', 'Romania', 'India', 'Japan', 'Germany',
       'Brazil', 'Singapore', 'Portugal', 'United Kingdom', 'France',
       'Spain', 'Vietnam', 'Malaysia', 'Mexico', 'Hungary', 'Turkey',
       'Austria', 'Slovenia', nan, 'Serbia', 'Australia', 'Slovakia',
       'Czechia', 'Lithuania', 'Ireland', 'Poland', 'Belgium', 'Colombia',
       None, 'Indonesia', 'Sweden', 'Thailand', 'Netherlands', 'Armenia',
       'Egypt', 'Greece', 'Ghana', 'Canada', 'Morocco', 'Denmark',
       'New Zealand', 'Kazakhstan', 'Italy', 'South Africa',
       'Switzerland', 'Argentina', 'Philippines', 'Costa Rica',
       'Guatemala', 'Georgia', 'Bulgaria', 'South Korea', 'Sri Lanka',
       'Chile', 'Panama', 'United Arab Emirates', 'Ukraine', 'Ecuador',
       'Finland', 'Chad', 'Mali', 'Norway', 'Taiwan', 'Nigeria', 'Peru'],
      dtype=object)

In [32]:
anjay[anjay['region'] == 'Southern Africa']

,city,state,zip_code,country,region,continent,fuzzy_match
898,None,None,None,South Africa,Southern Africa,None,False
1648,Brits,None,None,South Africa,Southern Africa,None,False
2126,Brits,None,None,South Africa,Southern Africa,None,False
2426,Brits,None,None,South Africa,Southern Africa,None,False
2723,None,None,None,South Africa,Southern Africa,None,False
4482,Brits,None,None,South Africa,Southern Africa,None,False
5490,Brits,None,None,South Africa,Southern Africa,None,False
5491,None,None,None,South Africa,Southern Africa,None,False
6645,Brits,None,None,South Africa,Southern Africa,None,False
8116,Johannesburg,None,None,South Africa,Southern Africa,None,False


In [33]:
anjay[anjay['country']=='United States'].head()

,city,state,zip_code,country,region,continent,fuzzy_match
0,None,Indiana,None,United States,None,None,False
1,None,Delaware,None,United States,None,None,False
6,Lincolnton,North Carolina,28092,United States,None,None,False
9,Charleston,South Carolina,29418,United States,None,None,False
10,None,Tennessee,None,United States,None,None,False


In [34]:
anjay[anjay['region'] == 'Northern America']

,city,state,zip_code,country,region,continent,fuzzy_match
465,None,None,None,Japan,Northern America,None,False
1783,None,None,None,None,Northern America,None,False
4144,Chicago,Illinois,60061,United States,Northern America,None,False
4218,None,None,None,None,Northern America,None,True
4352,None,None,None,None,Northern America,None,False
7003,None,None,None,Japan,Northern America,None,False
7785,None,None,None,None,Northern America,None,False
8381,None,None,None,None,Northern America,None,False
9862,None,None,None,None,Northern America,North America,True


#### mapping for `continent`

In [35]:
continent_mapping = {
    'United States': 'North America',
    'Canada': 'North America',
    'Brazil': 'South America',
    'Argentina': 'South America',
    'Mexico': 'North America',
    'China': 'Asia',
    'India': 'Asia',
    'Japan': 'Asia',
    'South Korea': 'Asia',
    'Vietnam': 'Asia',
    'Malaysia': 'Asia',
    'Indonesia': 'Asia',
    'Singapore': 'Asia',
    'Turkey': 'Asia',
    'Egypt': 'Africa',
    'South Africa': 'Africa',
    'Nigeria': 'Africa',
    'Ghana': 'Africa',
    'Morocco': 'Africa',
    'Germany': 'Europe',
    'France': 'Europe',
    'United Kingdom': 'Europe',
    'Spain': 'Europe',
    'Italy': 'Europe',
    'Netherlands': 'Europe',
    'Sweden': 'Europe',
    'Belgium': 'Europe',
    'Poland': 'Europe',
    'Czechia': 'Europe',
    'Austria': 'Europe',
    'Romania': 'Europe',
    'Hungary': 'Europe',
    'Slovakia': 'Europe',
    'Ireland': 'Europe',
    'Portugal': 'Europe',
    'Finland': 'Europe',
    'Denmark': 'Europe',
    'Slovenia': 'Europe',
    'Bulgaria': 'Europe',
    'Georgia': 'Asia',
    'Armenia': 'Asia',
    'Kazakhstan': 'Asia',
    'Thailand': 'Asia',
    'Philippines': 'Asia',
    'Chile': 'South America',
    'Colombia': 'South America',
    'Guatemala': 'North America',
    'Costa Rica': 'North America',
    'New Zealand': 'Oceania',
    'Australia': 'Oceania',
    'Taiwan': 'Asia',
    'United Arab Emirates': 'Asia',
    'Ukraine': 'Europe',
    'Ecuador': 'South America',
    'Chad': 'Africa',
    'Mali': 'Africa',
    'Norway': 'Europe',
    'Switzerland': 'Europe',
    'Greece': 'Europe',
    'Sri Lanka': 'Asia',
    'Vietnam': 'Asia',
    'Turkey': 'Asia',
    'Peru': 'South America',
    'Panama': 'North America',
    'Serbia' : 'Europe',
    'Lithuania': 'Europe'

}

In [36]:
def fill_continent(row):
    if pd.isna(row['country']) or row['country'] == '':
        return None
    return continent_mapping.get(row['country'], None)


anjay['continent'] = anjay.apply(fill_continent, axis=1)


In [37]:
anjay['continent'].unique()

array(['North America', 'Asia', 'Europe', 'South America', None,
       'Oceania', 'Africa'], dtype=object)

In [38]:
anjay['continent'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 9919 entries, 0 to 9918
Series name: continent
Non-Null Count  Dtype 
--------------  ----- 
9449 non-null   object
dtypes: object(1)
memory usage: 77.6+ KB


In [39]:
anjay['continent'].value_counts()

continent
Europe           3273
North America    3117
Asia             2521
South America     440
Africa             60
Oceania            38
Name: count, dtype: int64

In [40]:
anjay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9919 entries, 0 to 9918
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   city         6281 non-null   object
 1   state        2450 non-null   object
 2   zip_code     1045 non-null   object
 3   country      9449 non-null   object
 4   region       21 non-null     object
 5   continent    9449 non-null   object
 6   fuzzy_match  9503 non-null   object
dtypes: object(7)
memory usage: 542.6+ KB


#### Mapping for column `Region`

In [41]:
region_mapping = {
    'United States': 'North America',
    'China': 'East Asia',
    'Romania': 'Europe',
    'India': 'South Asia',
    'Japan': 'East Asia',
    'Germany': 'Europe',
    'Brazil': 'South America',
    'Singapore': 'Southeast Asia',
    'Portugal': 'Europe',
    'United Kingdom': 'Europe',
    'France': 'Europe',
    'Spain': 'Europe',
    'Vietnam': 'Southeast Asia',
    'Malaysia': 'Southeast Asia',
    'Mexico': 'North America',
    'Hungary': 'Europe',
    'Turkey': 'Middle East',
    'Austria': 'Europe',
    'Slovenia': 'Europe',
    'Serbia': 'Europe',
    'Australia': 'Oceania',
    'Slovakia': 'Europe',
    'Czechia': 'Europe',
    'Lithuania': 'Europe',
    'Ireland': 'Europe',
    'Poland': 'Europe',
    'Belgium': 'Europe',
    'Colombia': 'South America',
    'Indonesia': 'Southeast Asia',
    'Sweden': 'Europe',
    'Thailand': 'Southeast Asia',
    'Netherlands': 'Europe',
    'Armenia': 'Middle East',
    'Egypt': 'Middle East',
    'Greece': 'Europe',
    'Ghana': 'Africa',
    'Canada': 'North America',
    'Morocco': 'Africa',
    'Denmark': 'Europe',
    'New Zealand': 'Oceania',
    'Kazakhstan': 'Central Asia',
    'Italy': 'Europe',
    'South Africa': 'Africa',
    'Switzerland': 'Europe',
    'Argentina': 'South America',
    'Philippines': 'Southeast Asia',
    'Costa Rica': 'Central America',
    'Guatemala': 'Central America',
    'Georgia': 'Europe',
    'Bulgaria': 'Europe',
    'South Korea': 'East Asia',
    'Sri Lanka': 'South Asia',
    'Chile': 'South America',
    'Panama': 'Central America',
    'United Arab Emirates': 'Middle East',
    'Ukraine': 'Europe',
    'Ecuador': 'South America',
    'Finland': 'Europe',
    'Chad': 'Africa',
    'Mali': 'Africa',
    'Norway': 'Europe',
    'Taiwan': 'East Asia',
    'Nigeria': 'Africa',
    'Peru': 'South America'
}

In [42]:
anjay['region'] = anjay['country'].map(region_mapping)

#### Mapping column `State`

In [43]:
city_to_state = {
    'Phoenix': 'Arizona',
    'Los Angeles': 'California',
    'Austin': 'Texas',
    'Boston': 'Massachusetts',
    'Atlanta': 'Georgia',
    'Charlotte': 'North Carolina',
    'Dallas': 'Texas',
    'Denver': 'Colorado',
    'Miami': 'Florida',
    'New York': 'New York',
    'Chicago': 'Illinois',
    'Houston': 'Texas',
    'San Diego': 'California',
    'Seattle': 'Washington',
    'San Francisco': 'California',
    'Tampa': 'Florida',
    'Detroit': 'Michigan',
    'Washington': 'District of Columbia',
    'Baltimore': 'Maryland',
    'Raleigh': 'North Carolina',
}

In [44]:
anjay['state'] = anjay['city'].map(city_to_state)
anjay['state'].fillna(anjay['country'], inplace=True)

In [45]:
anjay['state'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 9919 entries, 0 to 9918
Series name: state
Non-Null Count  Dtype 
--------------  ----- 
9449 non-null   object
dtypes: object(1)
memory usage: 77.6+ KB


## Merging dataframe ___ and job_posting 

In [46]:
columns_to_update = ['city', 'state', 'zip_code','country', 'region', 'continent', 'fuzzy_match']

job_posting.update(anjay[columns_to_update])

job_posting.head()

,Website Domain,Job Opening Title,Job Opening URL,First Seen At,Last Seen At,Location,Category,Seniority,Keywords,Description,...,O*NET Code,O*NET Family,O*NET Occupation Name,city,state,zip_code,country,region,continent,fuzzy_match
0,bosch.com,IN_RBAI_Assistant Manager_Dispensing Process E...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-05-29T19:59:45Z,2024-07-31T14:35:44Z,"Indiana, United States","engineering, management, support",manager,NaN,**IN\_RBAI\_Assistant Manager\_Dispensing Proc...,...,43-1011.00,Office and Administrative Support,First-Line Supervisors of Office and Administr...,None,United States,None,United States,North America,North America,False
1,bosch.com,Professional Internship: Hardware Development ...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-05-04T01:00:12Z,2024-07-29T17:46:16Z,"Delaware, United States",internship,non_manager,Scrum,**Professional Internship: Hardware Developmen...,...,17-2061.00,Architecture and Engineering,Computer Hardware Engineers,None,United States,None,United States,North America,North America,False
2,zf.com,Process Expert BMS Production,https://jobs.zf.com/job/Shenyang-Process-Exper...,2024-04-19T06:47:24Z,2024-05-16T02:25:08Z,China,engineering,non_manager,SAP,ZF is a global technology company supplying sy...,...,51-9141.00,Production,Semiconductor Processing Technicians,None,China,None,China,East Asia,Asia,False
3,bosch.com,DevOps Developer with Python for ADAS Computin...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-08-16T10:20:37Z,2024-08-22T11:14:49Z,Romania,"information_technology, software_development",non_manager,"GitHub, Jenkins, Growth, C++, Linux, Python, M...",**DevOps Developer with Python for ADAS Comput...,...,15-1252.00,Computer and Mathematical,Software Developers,None,Romania,None,Romania,Europe,Europe,False
4,bosch.com,Senior Engineer Sales - Video Systems and Solu...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-07-01T17:31:20Z,2024-08-01T05:11:33Z,India,"engineering, sales",non_manager,Business Development,**Senior Engineer Sales - Video Systems and So...,...,41-9031.00,Sales and Related,Sales Engineers,None,India,None,India,South Asia,Asia,False


#### I decided to remove the `zip_code` column because the data has a very high missing value and there is no column and approach that is strong enough to impute the column.

In [47]:
job_posting_missing_city = job_posting[job_posting['city'].isna()]

job_posting_missing_city[['Location']]

,Location
0,"Indiana, United States"
1,"Delaware, United States"
2,China
3,Romania
4,India
...,...
9905,Slovenia
9906,Poland
9907,"Indiana, United States"
9915,"Indiana, United States"


## drop column `Zip_Code`

In [48]:
job_posting.drop('zip_code', axis=1, inplace=True)

In [49]:
null_values = job_posting['Category'].isnull().sum().sum()
not_null_values = job_posting['Category'].notnull().sum().sum()
total_data = job_posting['Category'].size
total_missing = job_posting['Category'].isnull().sum().sum()

percentage = (total_missing / total_data) * 100

print(f'Null values in data\t: {null_values}')
print(f'Not null values in data\t: {not_null_values}')
print(f'Dimension of Data\t: {total_data}')
print(f'Percentage missing values : {percentage:.2f}%')

Null values in data	: 1669
Not null values in data	: 8250
Dimension of Data	: 9919
Percentage missing values : 16.83%


In [50]:
# # Asumsikan df adalah DataFrame Anda
# contingency_table = pd.crosstab(
#     job_posting['Description'], job_posting['Keywords'])
# print(contingency_table)

In [51]:
# from scipy.stats import chi2_contingency

# chi2, p, dof, expected = chi2_contingency(contingency_table)
# print("p-value:", p)

In [52]:
# import seaborn as sns

# sns.countplot(x='Category', hue='Keywords', data=job_posting)

### covert columns `First Seen At` &	`Last Seen At` to DateTime

In [53]:
job_posting['First Seen At'] = pd.to_datetime(job_posting['First Seen At'])
job_posting['Last Seen At'] = pd.to_datetime(job_posting['Last Seen At'])
job_posting['Job Last Processed At'] = pd.to_datetime(job_posting['Job Last Processed At'])

In [54]:
job_posting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9919 entries, 0 to 9918
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   Website Domain         9919 non-null   object             
 1   Job Opening Title      9919 non-null   object             
 2   Job Opening URL        9919 non-null   object             
 3   First Seen At          9919 non-null   datetime64[ns, UTC]
 4   Last Seen At           9919 non-null   datetime64[ns, UTC]
 5   Location               9508 non-null   object             
 6   Category               8250 non-null   object             
 7   Seniority              9919 non-null   object             
 8   Keywords               7646 non-null   object             
 9   Description            9807 non-null   object             
 10  Salary                 576 non-null    object             
 11  Contract Types         8004 non-null   object           

### column `Job Opening Title` and `Category` are have correlation, so i decide to imputation ___Category___ based on ___Job Opening Title___

In [55]:
job_posting[['Job Opening Title','Category']].head()

,Job Opening Title,Category
0,IN_RBAI_Assistant Manager_Dispensing Process E...,"engineering, management, support"
1,Professional Internship: Hardware Development ...,internship
2,Process Expert BMS Production,engineering
3,DevOps Developer with Python for ADAS Computin...,"information_technology, software_development"
4,Senior Engineer Sales - Video Systems and Solu...,"engineering, sales"


In [56]:
df_with_category = job_posting[job_posting['Category'].notna()]
df_without_category = job_posting[job_posting['Category'].isna()]

vectorizer = TfidfVectorizer()
X_with_category = vectorizer.fit_transform(
    df_with_category['Job Opening Title'])

knn = KNeighborsClassifier(n_neighbors=3)  
knn.fit(X_with_category, df_with_category['Category'])

X_without_category = vectorizer.transform(
    df_without_category['Job Opening Title'])

predicted_categories = knn.predict(X_without_category)

job_posting.loc[job_posting['Category'].isna(
), 'Category'] = predicted_categories

# Tampilkan hasil
print(job_posting)

     Website Domain                                  Job Opening Title  \
0         bosch.com  IN_RBAI_Assistant Manager_Dispensing Process E...   
1         bosch.com  Professional Internship: Hardware Development ...   
2            zf.com                      Process Expert BMS Production   
3         bosch.com  DevOps Developer with Python for ADAS Computin...   
4         bosch.com  Senior Engineer Sales - Video Systems and Solu...   
...             ...                                                ...   
9914      bosch.com                                 Apprentice Program   
9915      bosch.com  Environmental, Health and Safety (EHS) Special...   
9916      bosch.com          Technology & Quality Expert in Purchasing   
9917      bosch.com  Extracurricular Internship: Test Management (f...   
9918         zf.com                      System Qualification Engineer   

                                        Job Opening URL  \
0     https://jobs.smartrecruiters.com/BoschGroup/74

In [57]:
df_with_category = job_posting[job_posting['Keywords'].notna()]
df_without_category = job_posting[job_posting['Keywords'].isna()]

vectorizer = TfidfVectorizer()
X_with_category = vectorizer.fit_transform(
    df_with_category['Job Opening Title'])

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_with_category, df_with_category['Keywords'])

X_without_category = vectorizer.transform(
    df_without_category['Job Opening Title'])

predicted_categories = knn.predict(X_without_category)

job_posting.loc[job_posting['Keywords'].isna(
), 'Keywords'] = predicted_categories

# Tampilkan hasil
print(job_posting)

     Website Domain                                  Job Opening Title  \
0         bosch.com  IN_RBAI_Assistant Manager_Dispensing Process E...   
1         bosch.com  Professional Internship: Hardware Development ...   
2            zf.com                      Process Expert BMS Production   
3         bosch.com  DevOps Developer with Python for ADAS Computin...   
4         bosch.com  Senior Engineer Sales - Video Systems and Solu...   
...             ...                                                ...   
9914      bosch.com                                 Apprentice Program   
9915      bosch.com  Environmental, Health and Safety (EHS) Special...   
9916      bosch.com          Technology & Quality Expert in Purchasing   
9917      bosch.com  Extracurricular Internship: Test Management (f...   
9918         zf.com                      System Qualification Engineer   

                                        Job Opening URL  \
0     https://jobs.smartrecruiters.com/BoschGroup/74

In [58]:
job_posting[job_posting['Job Opening URL']=='closed']

,Website Domain,Job Opening Title,Job Opening URL,First Seen At,Last Seen At,Location,Category,Seniority,Keywords,Description,...,Job Last Processed At,O*NET Code,O*NET Family,O*NET Occupation Name,city,state,country,region,continent,fuzzy_match


In [59]:
# job_posting.iloc[:, :12]

In [60]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)

In [61]:
df_with_category = job_posting[job_posting['Contract Types'].notna()]
df_without_category = job_posting[job_posting['Contract Types'].isna()]

vectorizer = TfidfVectorizer()
X_with_category = vectorizer.fit_transform(
    df_with_category['Job Opening Title'])

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_with_category, df_with_category['Contract Types'])

X_without_category = vectorizer.transform(
    df_without_category['Job Opening Title'])

predicted_categories = knn.predict(X_without_category)

job_posting.loc[job_posting['Contract Types'].isna(
), 'Contract Types'] = predicted_categories

# Tampilkan hasil
print(job_posting)

     Website Domain                                  Job Opening Title  \
0         bosch.com  IN_RBAI_Assistant Manager_Dispensing Process E...   
1         bosch.com  Professional Internship: Hardware Development ...   
2            zf.com                      Process Expert BMS Production   
3         bosch.com  DevOps Developer with Python for ADAS Computin...   
4         bosch.com  Senior Engineer Sales - Video Systems and Solu...   
...             ...                                                ...   
9914      bosch.com                                 Apprentice Program   
9915      bosch.com  Environmental, Health and Safety (EHS) Special...   
9916      bosch.com          Technology & Quality Expert in Purchasing   
9917      bosch.com  Extracurricular Internship: Test Management (f...   
9918         zf.com                      System Qualification Engineer   

                                        Job Opening URL  \
0     https://jobs.smartrecruiters.com/BoschGroup/74

In [62]:
job_posting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9919 entries, 0 to 9918
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   Website Domain         9919 non-null   object             
 1   Job Opening Title      9919 non-null   object             
 2   Job Opening URL        9919 non-null   object             
 3   First Seen At          9919 non-null   datetime64[ns, UTC]
 4   Last Seen At           9919 non-null   datetime64[ns, UTC]
 5   Location               9508 non-null   object             
 6   Category               9919 non-null   object             
 7   Seniority              9919 non-null   object             
 8   Keywords               9919 non-null   object             
 9   Description            9807 non-null   object             
 10  Salary                 576 non-null    object             
 11  Contract Types         9919 non-null   object           

#### Mapping for column `City`

In [63]:
anjay = job_posting.copy()

In [64]:
def split_location(location):
    if isinstance(location, str) and ',' in location:
        return location.split(', ', 1)
    return ['', '']

anjay['city'] = anjay['Location'].apply(split_location)
city_mapping = anjay[anjay['city'].notna()]
city_mapping[['city', 'country']] = pd.DataFrame(city_mapping['city'].tolist(), index=city_mapping.index)



In [65]:
city_mapping.iloc[:, 15:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9919 entries, 0 to 9918
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   O*NET Code             9916 non-null   object
 1   O*NET Family           9916 non-null   object
 2   O*NET Occupation Name  9916 non-null   object
 3   city                   9919 non-null   object
 4   state                  9449 non-null   object
 5   country                9919 non-null   object
 6   region                 9455 non-null   object
 7   continent              9482 non-null   object
 8   fuzzy_match            9503 non-null   object
dtypes: object(9)
memory usage: 697.6+ KB


In [66]:
null_values = city_mapping['region'].isnull().sum().sum()
not_null_values = city_mapping['region'].notnull().sum().sum()
total_data = city_mapping['region'].size
total_missing = city_mapping['region'].isnull().sum().sum()

percentage = (total_missing / total_data) * 100

print(f'Null values in data\t: {null_values}')
print(f'Not null values in data\t: {not_null_values}')
print(f'Dimension of Data\t: {total_data}')
print(f'Percentage missing values : {percentage:.2f}%')

Null values in data	: 464
Not null values in data	: 9455
Dimension of Data	: 9919
Percentage missing values : 4.68%


In [67]:
city_mapping['city']

0          Indiana
1         Delaware
2                 
3                 
4                 
           ...    
9914    Charleston
9915       Indiana
9916              
9917        Aveiro
9918       Jiaxing
Name: city, Length: 9919, dtype: object

## mergin ___ with job_posting

In [68]:
columns_to_update = ['city']

job_posting.update(city_mapping[columns_to_update])

job_posting.head()

,Website Domain,Job Opening Title,Job Opening URL,First Seen At,Last Seen At,Location,Category,Seniority,Keywords,Description,...,Job Last Processed At,O*NET Code,O*NET Family,O*NET Occupation Name,city,state,country,region,continent,fuzzy_match
0,bosch.com,IN_RBAI_Assistant Manager_Dispensing Process E...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-05-29 19:59:45+00:00,2024-07-31 14:35:44+00:00,"Indiana, United States","engineering, management, support",manager,"C#, Python",**IN\_RBAI\_Assistant Manager\_Dispensing Proc...,...,2024-08-02 14:47:55+00:00,43-1011.00,Office and Administrative Support,First-Line Supervisors of Office and Administr...,Indiana,United States,United States,North America,North America,False
1,bosch.com,Professional Internship: Hardware Development ...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-05-04 01:00:12+00:00,2024-07-29 17:46:16+00:00,"Delaware, United States",internship,non_manager,Scrum,**Professional Internship: Hardware Developmen...,...,2024-07-31 17:50:07+00:00,17-2061.00,Architecture and Engineering,Computer Hardware Engineers,Delaware,United States,United States,North America,North America,False
2,zf.com,Process Expert BMS Production,https://jobs.zf.com/job/Shenyang-Process-Exper...,2024-04-19 06:47:24+00:00,2024-05-16 02:25:08+00:00,China,engineering,non_manager,SAP,ZF is a global technology company supplying sy...,...,2024-05-18 02:32:04+00:00,51-9141.00,Production,Semiconductor Processing Technicians,,China,China,East Asia,Asia,False
3,bosch.com,DevOps Developer with Python for ADAS Computin...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-08-16 10:20:37+00:00,2024-08-22 11:14:49+00:00,Romania,"information_technology, software_development",non_manager,"GitHub, Jenkins, Growth, C++, Linux, Python, M...",**DevOps Developer with Python for ADAS Comput...,...,2024-08-23 00:33:30+00:00,15-1252.00,Computer and Mathematical,Software Developers,,Romania,Romania,Europe,Europe,False
4,bosch.com,Senior Engineer Sales - Video Systems and Solu...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-07-01 17:31:20+00:00,2024-08-01 05:11:33+00:00,India,"engineering, sales",non_manager,Business Development,**Senior Engineer Sales - Video Systems and So...,...,2024-08-02 19:03:16+00:00,41-9031.00,Sales and Related,Sales Engineers,,India,India,South Asia,Asia,False


In [69]:
analysis_job_status_closed = city_mapping[city_mapping['Job Status'] == 'closed'][[
    'First Seen At', 'Last Seen At', 'Job Status', 'Job Last Processed At']]

analysis_job_status_closed

,First Seen At,Last Seen At,Job Status,Job Last Processed At
0,2024-05-29 19:59:45+00:00,2024-07-31 14:35:44+00:00,closed,2024-08-02 14:47:55+00:00
1,2024-05-04 01:00:12+00:00,2024-07-29 17:46:16+00:00,closed,2024-07-31 17:50:07+00:00
2,2024-04-19 06:47:24+00:00,2024-05-16 02:25:08+00:00,closed,2024-05-18 02:32:04+00:00
3,2024-08-16 10:20:37+00:00,2024-08-22 11:14:49+00:00,closed,2024-08-23 00:33:30+00:00
4,2024-07-01 17:31:20+00:00,2024-08-01 05:11:33+00:00,closed,2024-08-02 19:03:16+00:00
...,...,...,...,...
9906,2024-04-18 00:35:17+00:00,2024-05-02 01:39:39+00:00,closed,2024-05-04 01:44:14+00:00
9907,2024-07-01 17:31:20+00:00,2024-07-07 17:57:32+00:00,closed,2024-07-09 18:05:48+00:00
9908,2024-07-18 10:06:39+00:00,2024-08-25 17:51:17+00:00,closed,2024-08-27 18:00:18+00:00
9910,2024-04-22 23:30:12+00:00,2024-05-07 00:33:15+00:00,closed,2024-05-09 00:35:46+00:00


In [70]:
analysis_job_status_null = city_mapping[city_mapping['Job Status'] != 'closed'][[
    'First Seen At', 'Last Seen At', 'Job Status', 'Job Last Processed At']]
analysis_job_status_null

,First Seen At,Last Seen At,Job Status,Job Last Processed At
8,2024-08-23 23:49:15+00:00,2024-09-03 00:28:12+00:00,NaN,2024-09-03 00:28:12+00:00
11,2024-04-04 07:41:09+00:00,2024-09-03 02:22:43+00:00,NaN,2024-09-03 02:22:43+00:00
12,2024-08-08 19:44:45+00:00,2024-09-04 00:28:08+00:00,NaN,2024-09-04 00:28:08+00:00
13,2024-08-04 00:40:05+00:00,2024-08-31 01:41:45+00:00,NaN,2024-08-04 00:53:42+00:00
27,2024-05-15 03:34:45+00:00,2024-09-02 23:30:44+00:00,NaN,2024-09-02 23:30:44+00:00
...,...,...,...,...
9913,2024-05-14 16:19:51+00:00,2024-06-11 21:56:20+00:00,NaN,2024-05-14 16:31:43+00:00
9915,2024-08-29 01:16:54+00:00,2024-09-04 05:08:32+00:00,NaN,2024-09-04 02:01:08+00:00
9916,2024-07-19 14:36:25+00:00,2024-09-03 19:39:05+00:00,NaN,2024-09-03 19:39:05+00:00
9917,2024-08-05 18:17:27+00:00,2024-09-03 22:46:37+00:00,NaN,2024-09-03 22:46:37+00:00


In [71]:
peler = job_posting.copy()

In [72]:
def update_job_status(df, threshold_days=30):
    current_date = pd.Timestamp.now(tz='UTC')

    def determine_status(row):
        last_seen = pd.to_datetime(row['Last Seen At'], utc=True)
        days_since_last_seen = (current_date - last_seen).days

        if days_since_last_seen > threshold_days:
            return 'closed'
        else:
            return 'open'

    df['Job Status'] = df.apply(
        lambda row: determine_status(row), axis=1)
    return df

updated_df = update_job_status(peler)

In [73]:
updated_df['Job Status']

0       closed
1       closed
2       closed
3       closed
4       closed
         ...  
9914    closed
9915      open
9916      open
9917      open
9918      open
Name: Job Status, Length: 9919, dtype: object

In [74]:
columns_to_update = ['Job Status']

job_posting.update(updated_df[columns_to_update])

job_posting.head()

,Website Domain,Job Opening Title,Job Opening URL,First Seen At,Last Seen At,Location,Category,Seniority,Keywords,Description,...,Job Last Processed At,O*NET Code,O*NET Family,O*NET Occupation Name,city,state,country,region,continent,fuzzy_match
0,bosch.com,IN_RBAI_Assistant Manager_Dispensing Process E...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-05-29 19:59:45+00:00,2024-07-31 14:35:44+00:00,"Indiana, United States","engineering, management, support",manager,"C#, Python",**IN\_RBAI\_Assistant Manager\_Dispensing Proc...,...,2024-08-02 14:47:55+00:00,43-1011.00,Office and Administrative Support,First-Line Supervisors of Office and Administr...,Indiana,United States,United States,North America,North America,False
1,bosch.com,Professional Internship: Hardware Development ...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-05-04 01:00:12+00:00,2024-07-29 17:46:16+00:00,"Delaware, United States",internship,non_manager,Scrum,**Professional Internship: Hardware Developmen...,...,2024-07-31 17:50:07+00:00,17-2061.00,Architecture and Engineering,Computer Hardware Engineers,Delaware,United States,United States,North America,North America,False
2,zf.com,Process Expert BMS Production,https://jobs.zf.com/job/Shenyang-Process-Exper...,2024-04-19 06:47:24+00:00,2024-05-16 02:25:08+00:00,China,engineering,non_manager,SAP,ZF is a global technology company supplying sy...,...,2024-05-18 02:32:04+00:00,51-9141.00,Production,Semiconductor Processing Technicians,,China,China,East Asia,Asia,False
3,bosch.com,DevOps Developer with Python for ADAS Computin...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-08-16 10:20:37+00:00,2024-08-22 11:14:49+00:00,Romania,"information_technology, software_development",non_manager,"GitHub, Jenkins, Growth, C++, Linux, Python, M...",**DevOps Developer with Python for ADAS Comput...,...,2024-08-23 00:33:30+00:00,15-1252.00,Computer and Mathematical,Software Developers,,Romania,Romania,Europe,Europe,False
4,bosch.com,Senior Engineer Sales - Video Systems and Solu...,https://jobs.smartrecruiters.com/BoschGroup/74...,2024-07-01 17:31:20+00:00,2024-08-01 05:11:33+00:00,India,"engineering, sales",non_manager,Business Development,**Senior Engineer Sales - Video Systems and So...,...,2024-08-02 19:03:16+00:00,41-9031.00,Sales and Related,Sales Engineers,,India,India,South Asia,Asia,False


In [75]:
job_posting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9919 entries, 0 to 9918
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   Website Domain         9919 non-null   object             
 1   Job Opening Title      9919 non-null   object             
 2   Job Opening URL        9919 non-null   object             
 3   First Seen At          9919 non-null   datetime64[ns, UTC]
 4   Last Seen At           9919 non-null   datetime64[ns, UTC]
 5   Location               9508 non-null   object             
 6   Category               9919 non-null   object             
 7   Seniority              9919 non-null   object             
 8   Keywords               9919 non-null   object             
 9   Description            9807 non-null   object             
 10  Salary                 576 non-null    object             
 11  Contract Types         9919 non-null   object           

## delete column `Salary`

In [76]:
job_posting.drop('Salary', axis=1, inplace=True)

In [77]:
asaas = job_posting.copy()
asaas.dropna(inplace=True)

In [78]:
asaas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9385 entries, 0 to 9918
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   Website Domain         9385 non-null   object             
 1   Job Opening Title      9385 non-null   object             
 2   Job Opening URL        9385 non-null   object             
 3   First Seen At          9385 non-null   datetime64[ns, UTC]
 4   Last Seen At           9385 non-null   datetime64[ns, UTC]
 5   Location               9385 non-null   object             
 6   Category               9385 non-null   object             
 7   Seniority              9385 non-null   object             
 8   Keywords               9385 non-null   object             
 9   Description            9385 non-null   object             
 10  Contract Types         9385 non-null   object             
 11  Job Status             9385 non-null   object             
 1

In [79]:
null_values = job_posting.isnull().sum().sum()
not_null_values = job_posting.notnull().sum().sum()
total_data = job_posting.size
total_missing = job_posting.isnull().sum().sum()

percentage = (total_missing / total_data) * 100

print(f'Null values in data\t: {null_values}')
print(f'Not null values in data\t: {not_null_values}')
print(f'Dimension of Data\t: {total_data}')
print(f'Percentage missing values : {percentage:.2f}%')

Null values in data	: 2791
Not null values in data	: 225346
Dimension of Data	: 228137
Percentage missing values : 1.22%


In [95]:
job_posting.dropna(inplace=True)

In [97]:
null_values = job_posting.isnull().sum().sum()
not_null_values = job_posting.notnull().sum().sum()
total_data = job_posting.size
total_missing = job_posting.isnull().sum().sum()

percentage = (total_missing / total_data) * 100

print(f'Null values in data\t: {null_values}')
print(f'Not null values in data\t: {not_null_values}')
print(f'Dimension of Data\t: {total_data}')
print(f'Percentage missing values : {percentage:.2f}%')

Null values in data	: 0
Not null values in data	: 215855
Dimension of Data	: 215855
Percentage missing values : 0.00%


In [81]:
# job_posting.drop('zip_code', axis=1, inplace=True)

In [82]:
# job_posting.head()

In [83]:
# anjay[anjay['continent'].isna()]

In [84]:
# # Menampilkan baris yang memiliki 'continent' bernilai None (atau NaN)
# missing_continent = anjay[anjay['continent'].isna()]

# # Cetak daftar negara yang tidak memiliki continent
# print(missing_continent[['country']].drop_duplicates())

In [85]:
# job_posting.drop('Ticker', axis=1, inplace=True)
# job_posting.columns

In [86]:
# df = job_posting.loc[:, 'salary_low':'salary_time_unit']
# df.info()

In [87]:
# for value in anjay['city'].unique():
#   print(value)

In [88]:
# numbers = anjay['city'].tolist()
# numbers = numbers
# print(type(numbers))

In [89]:
# def get_unique_numbers(numbers):

#     list_of_unique_numbers = []

#     unique_numbers = set(numbers)

#     for number in unique_numbers:
#         list_of_unique_numbers.append(number)

#     return list_of_unique_numbers


# print(get_unique_numbers(numbers))

In [90]:
# job_posting.loc['salary_low':'salary_timne_unit'].info()

In [91]:
# filtered_df = job_posting[
#     job_posting['salary_low'].notna() &
#     job_posting['salary_high'].notna() &
#     job_posting['salary_currency'].notna() &
#     job_posting['salary_low_usd'].notna() &
#     job_posting['salary_high_usd'].notna()
# ]
# filtered_df

In [92]:
# filter_domai = filtered_df[job_posting['Website Domain'] == 'bosch.com']
# filter_domai[:]

In [93]:
# pd.set_option('display.max_rows', None)

In [94]:
# filter_domain = filter_domai[(filter_domai['Website Domain'] == 'bosch.com') &
#                    (filter_domai['salary_low'] == filter_domai['salary_high'])]
# sorted_filter_domain = filter_domain.sort_values(
#     by='salary_low', ascending=True)
# sorted_filter_domain